In [1]:
%load_ext autoreload
%autoreload 2

from statsmodels.base.model import GenericLikelihoodModel

import numpy as np
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import sys

from scipy.stats import multivariate_normal

sys.path.append("../")
import vuong_tests5

In [2]:
class JointNormal1(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([params[0], 0.0], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())
    
    
class JointNormal2(GenericLikelihoodModel):
    
    def loglikeobs(self, params):
        data = np.concatenate([[self.endog],self.exog.transpose()],axis=0)
        mult_rv = stats.multivariate_normal([0.0, params[0]], [[1,0],[0,1]])
        return mult_rv.logpdf(data.transpose())


def setup_shi(yn,xn):
    # model 1 grad, etc.
    nobs = yn.shape[0]
    model1_param = np.array([yn.mean()])
    model2_param = np.array([xn.mean()])
    
    model1_deriv = JointNormal1(yn,xn)
    ll1 = model1_deriv.loglikeobs(model1_param)
    grad1 =  model1_deriv.score_obs(model1_param).reshape( (nobs,1) )
    hess1 = model1_deriv.hessian(model1_param)
    
    
    model2_deriv = JointNormal2(yn,xn)
    ll2 = model2_deriv.loglikeobs(model2_param)
    grad2 =  model2_deriv.score_obs(model2_param).reshape( (nobs,1) )  
    hess2 = model2_deriv.hessian(model2_param)
    
    return ll1,grad1,hess1,model1_param,ll2,grad2,hess2,model2_param

def gen_data(beta= 1.5, nobs=1000):
    cov = [[25, 0], [0, 1]]
    data = np.random.multivariate_normal([beta,beta], [[25,0],[0,1]],  nobs)
    return data[:,0],data[:,1],nobs

yn,xn,nobs = gen_data()
ll1,grad1,hess1,params1,ll2,grad2,hess2,params2 = setup_shi(yn,xn)
print(grad1.shape,hess1.shape)

(1000, 1) (1, 1)


# Ex 0

In [3]:
beta = 0

In [4]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 0.86 & 0.86 & 0.85 & 0.89   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.14 & 0.14 & 0.15 & 0.11   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([0.86, 0.  , 0.14]), array([0.86, 0.  , 0.14]), array([0.85, 0.  , 0.15]), array([0.89, 0.  , 0.11]), 15.36872829885135, 18.330598777914318, 23.3064673852329)


In [5]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 0.82 & 0.82 & 0.84 & 0.94   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.18 & 0.18 & 0.16 & 0.06   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([0.82, 0.  , 0.18]), array([0.82, 0.  , 0.18]), array([0.84, 0.  , 0.16]), array([0.94, 0.  , 0.06]), 13.12425114935521, 19.07244181239873, 20.43262069588012)


In [6]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 1.00 & 0.75 & 0.75 & 0.75 & 0.86   \\
Model 1 & 0.00 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.00 & 0.25 & 0.25 & 0.25 & 0.14   \\
\hline
\end{tabular}
(array([1., 0., 0.]), array([0.75, 0.  , 0.25]), array([0.75, 0.  , 0.25]), array([0.75, 0.  , 0.25]), array([0.86, 0.  , 0.14]), 10.705337880371497, 15.138964176378654, 18.7428297554722)


# Ex 1

In [7]:
beta = 1

In [8]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.87 & 0.83 & 0.81 & 0.82 & 0.98   \\
Model 1 & 0.00 & 0.01 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.13 & 0.16 & 0.19 & 0.18 & 0.02   \\
\hline
\end{tabular}
(array([0.87, 0.  , 0.13]), array([0.83, 0.01, 0.16]), array([0.81, 0.  , 0.19]), array([0.82, 0.  , 0.18]), array([0.98, 0.  , 0.02]), -2.9874914070416634, 84.07518435775069, 77.28645257785207)


In [9]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.89 & 0.83 & 0.83 & 0.84 & 1.00   \\
Model 1 & 0.01 & 0.03 & 0.01 & 0.02 & 0.00   \\
Model 2 & 0.10 & 0.14 & 0.16 & 0.14 & 0.00   \\
\hline
\end{tabular}
(array([0.89, 0.01, 0.1 ]), array([0.83, 0.03, 0.14]), array([0.83, 0.01, 0.16]), array([0.84, 0.02, 0.14]), array([1., 0., 0.]), -13.545564665075032, 120.21153154877882, 108.34182960809963)


In [10]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.93 & 0.93 & 0.93 & 0.95 & 1.00   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.00 & 0.00   \\
Model 2 & 0.06 & 0.06 & 0.06 & 0.05 & 0.00   \\
\hline
\end{tabular}
(array([0.93, 0.01, 0.06]), array([0.93, 0.01, 0.06]), array([0.93, 0.01, 0.06]), array([0.95, 0.  , 0.05]), array([1., 0., 0.]), -0.15353780325049243, 153.3601426617949, 159.45647309613966)


# Ex 2

In [11]:
beta = 1.5

In [12]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.95 & 0.94 & 0.94 & 1.00   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.01 & 0.00   \\
Model 2 & 0.03 & 0.04 & 0.05 & 0.05 & 0.00   \\
\hline
\end{tabular}
(array([0.96, 0.01, 0.03]), array([0.95, 0.01, 0.04]), array([0.94, 0.01, 0.05]), array([0.94, 0.01, 0.05]), array([1., 0., 0.]), 6.691989986442804, 107.93120648745811, 119.66678199796218)


In [13]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.96 & 0.97 & 0.94 & 1.00   \\
Model 1 & 0.01 & 0.00 & 0.00 & 0.00 & 0.00   \\
Model 2 & 0.04 & 0.04 & 0.03 & 0.06 & 0.00   \\
\hline
\end{tabular}
(array([0.95, 0.01, 0.04]), array([0.96, 0.  , 0.04]), array([0.97, 0.  , 0.03]), array([0.94, 0.  , 0.06]), array([1., 0., 0.]), 18.132945272194746, 143.28113541405065, 171.71986965741726)


In [14]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.97 & 0.95 & 0.98 & 0.98 & 1.00   \\
Model 1 & 0.02 & 0.03 & 0.01 & 0.01 & 0.00   \\
Model 2 & 0.01 & 0.02 & 0.01 & 0.01 & 0.00   \\
\hline
\end{tabular}
(array([0.97, 0.02, 0.01]), array([0.95, 0.03, 0.02]), array([0.98, 0.01, 0.01]), array([0.98, 0.01, 0.01]), array([1., 0., 0.]), 36.529033024461285, 240.90609196635, 244.30972402726414)


# Ex 3

In [15]:
beta = 2

In [16]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=250, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.95 & 0.94 & 0.92 & 0.94 & 0.99   \\
Model 1 & 0.02 & 0.01 & 0.02 & 0.01 & 0.00   \\
Model 2 & 0.03 & 0.05 & 0.06 & 0.05 & 0.01   \\
\hline
\end{tabular}
(array([0.95, 0.02, 0.03]), array([0.94, 0.01, 0.05]), array([0.92, 0.02, 0.06]), array([0.94, 0.01, 0.05]), array([0.99, 0.  , 0.01]), 9.326506378000952, 155.04394633485532, 160.25895058865012)


In [17]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=500, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi,trials=100)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.94 & 0.91 & 0.90 & 0.91 & 1.00   \\
Model 1 & 0.02 & 0.04 & 0.03 & 0.03 & 0.00   \\
Model 2 & 0.04 & 0.05 & 0.07 & 0.06 & 0.00   \\
\hline
\end{tabular}
(array([0.94, 0.02, 0.04]), array([0.91, 0.04, 0.05]), array([0.9 , 0.03, 0.07]), array([0.91, 0.03, 0.06]), array([1., 0., 0.]), 40.95885401894301, 232.39155204304345, 229.75937490530978)


In [18]:
setup_shi_ex  = lambda yn,xn: setup_shi(yn,xn,num_params=num_params)
gen_data_ex = lambda : gen_data(nobs=1000, beta =beta)
mc_out = vuong_tests5.monte_carlo(100,gen_data_ex,setup_shi)
vuong_tests5.print_mc(mc_out)
print(mc_out)

\begin{tabular}{|c|c|c|c|c|c|}
\hline
Model &  Normal & Bootstrap & Bootstrap-pt & Bootstrap-bc & Shi (2015) \\ \hline \hline
No selection & 0.96 & 0.95 & 0.95 & 0.95 & 1.00   \\
Model 1 & 0.01 & 0.01 & 0.01 & 0.02 & 0.00   \\
Model 2 & 0.03 & 0.04 & 0.04 & 0.03 & 0.00   \\
\hline
\end{tabular}
(array([0.96, 0.01, 0.03]), array([0.95, 0.01, 0.04]), array([0.95, 0.01, 0.04]), array([0.95, 0.02, 0.03]), array([1., 0., 0.]), -22.38897799515464, 314.4139043830383, 320.7248762667574)


In [19]:
beta = 3